In [13]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient

In [14]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '440'
start = '2019-04-01T00:00:00Z'
end = '2021-06-03T00:00:00Z'
mongo_org_collection = db[str(organization_id)]
escalation = ["roadie_escalations", "ubiquity_escalation"]

In [15]:
def get_dataset():
    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': {'$in':['web','email', 'api']},
        'original_item.created_at': {'$gte': start, '$lte': end},
        "original_item.tags": {"$nin": escalation}
    })
    
    file_path = os.path.expanduser('~/Downloads/rdie_non_esc.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            if count < 14000:
                i = ticket["original_item"]
                variable_labels = [x for x in escalation if x in i['tags']]
                j = {
                    "text_a": i["subject"],
                    "text_b": i["description"],
                    "created_at": i["created_at"],
                    "label": "no macro applied"
                    }
                    
                json.dump(j, f)
                _ = f.write('\n')
                count += 1
                if count % 100 == 0:
                    print(count)
            else:
                break

    print("total: ", count)


get_dataset()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
total:  14000
